# 컨텍스트가 포함된 프롬프트를 사용하여 베드락 사용 해보기

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

이 노트북에서는 고객 지원 엔지니어로부터 받은 고객 서비스 품질에 만족하지 못한 고객에게 이메일 응답을 생성하는 방법을 보여드립니다. 불만족한 고객으로부터 받은 실제 이메일의 내용을 제공하여 모델에 추가 컨텍스트를 제공합니다.

프롬프트에 추가 컨텍스트가 제공되므로 이 노트북의 Amazon Titan 대형 언어 모델에서 생성된 텍스트는 이전에 제로 샷 프롬프트를 통해 생성된 콘텐츠보다 품질과 관련성이 훨씬 더 우수합니다.

[LangChain](https://python.langchain.com/docs/get_started/introduction.html)은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 핵심은 다양한 구성 요소를 연결하여 고급 사용 사례를 생성함으로써 대규모 언어 모델을 보강할 수 있게 해줍니다.

이 노트북에서는 LangChain에서 제공하는 Bedrock API를 사용하겠습니다. 이 예제에서 사용된 프롬프트는 텍스트 생성 요청에 컨텍스트를 추가하기 위한 사용자 정의 LangChain 프롬프트 템플릿을 생성합니다. 

**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다.*

#### Context
이전 예제 `01_zero_shot_generation.ipynb`에서는 LangChain 프레임워크를 사용하여 Amazon Bedrock API와 통신하는 방법을 살펴봤습니다. 이 노트북에서는 유사한 사용 사례에 대해 LangChain 프레임워크를 활용하기 위해 `PromptTemplates`의 도움으로 조금 더 복잡성을 추가하려고 합니다. 프롬프트 템플릿`을 사용하면 나중에 정보로 채울 수 있는 일반 셸을 생성하고 다양한 시나리오에 따라 모델 출력을 얻을 수 있습니다.

이 노트북의 일부로 LangChain 프레임워크 내에서 Amazon Bedrock 통합의 사용과 'PromptTemplate'의 도움으로 텍스트를 생성하는 데 사용할 수 있는 방법을 살펴볼 것입니다.

#### Pattern
저희는 추가 예제를 제공하지 않고, 작업, 명령어 및 내부 모델에 대한 입력으로 구성된 입력을 Amazon Bedrock API의 LangChain 구현에 제공하여 출력을 생성하도록 할 것입니다. 이 글의 목적은 강력한 LLM이 어떻게 당면한 작업을 쉽게 이해하고 매력적인 출력을 생성하는지 보여드리기 위함입니다.

![](./images/bedrock_langchain.jpg)

#### Use case
Amazon Bedrock에서 모델의 생성 기능을 보여주기 위해 이메일 생성의 사용 사례를 살펴보겠습니다.

#### Persona
당신은 애니컴퍼니의 고객 서비스 매니저인 밥이고, 일부 고객이 고객 서비스에 만족하지 못하고 고객 지원 엔지니어가 제공하는 서비스에 대해 부정적인 피드백을 제공하고 있습니다. 이제 이러한 고객들에게 서비스 불만족에 대해 겸손하게 사과하고 신뢰를 회복하고 싶습니다. 이전 이메일 서신에서 얻은 고객의 정서에 맞춰 인간적이고 개인화된 이메일을 대량으로 생성하려면 LLM의 도움이 필요합니다.

#### Implementation
이 사용 사례를 충족하기 위해 고객의 이전 이메일을 기반으로 감사 메모가 포함된 이메일을 생성하는 방법을 보여드리겠습니다. 여기서는 Amazon Bedrock LangChain 통합을 사용하여 Amazon Titan Text Large 모델을 사용합니다.


## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되어 있는지 확인하고 베드락에 연결해야 합니다.

설정 방법과 ⚠️ **변경이 필요한지 여부**에 대한 자세한 내용은 [Bedrock boto3 설정 노트북](../00_Setup/setup.ipynb) 노트북을 참조하세요.

이 노트북에는 입력 프롬프트에서 토큰 수를 계산하는 데 사용할 [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) 라이브러리도 설치합니다.

In [2]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                            0.0.249
opensearch-py                        2.3.2


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww

In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## Bedrock LLM 모델 호출

llms에서 Bedrock 클래스의 인스턴스를 생성하는 것으로 시작하겠습니다. 여기에는 아마존 베드락에서 사용할 수 있는 모델의 ARN인 `model_id`가 필요합니다. 

선택적으로 이전에 생성한 boto3 클라이언트를 전달할 수 있으며, `temperature`, `topP`, `maxTokenCount` 또는 `stopSequences`와 같은 매개 변수를 보유할 수 있는 `model_kwargs`도 전달할 수 있습니다(매개 변수에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 탐색할 수 있음).

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델의 ID는 다음과 같습니다:

- amazon.titan-tg1-large
- ai21.j2-grande-instruct
- ai21.j2-jumbo-instruct
- anthropic.claude-instant-v1
- anthropic.claude-v1

모델마다 서로 다른 `model_kwargs`를 지원한다는 점에 유의하세요.

In [5]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'maxTokenCount':4096, 
                      "temperature":0.5,
                      #"topK":250,
                      "topP":1,
                      "stopSequences": ["||"]
                     }

textgen_llm = Bedrock(model_id = "amazon.titan-tg1-large",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## LangChain 사용자 지정 프롬프트 템플릿 만들기

프롬프트에 대한 템플릿을 생성하면 실행할 때마다 다른 입력 변수를 프롬프트에 전달할 수 있습니다. 이는 데이터베이스에서 가져올 수 있는 다양한 입력 변수를 사용하여 콘텐츠를 생성해야 할 때 유용합니다.

이전에는 프롬프트를 하드코딩했지만, 여러 고객이 비슷한 부정적인 피드백을 보내는 경우 이제 각 고객의 이메일을 사용하여 사과와 함께 응답하고 싶지만 응답을 약간 개인화하고자 할 수 있습니다. 다음 셀에서는 이러한 패턴을 달성하기 위해 `프롬프트 템플릿`을 만드는 방법을 살펴봅니다.

In [6]:
from langchain import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""Create an apology email from the Service Manager {customerServiceManager} to {customerName}. 
   in response to the following feedback that was received from the customer: {feedbackFromCustomer}.
   """
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [7]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Our prompt has 135 tokens


## 다시 호출하기

프롬프트 템플릿을 사용하여 호출하고 선별된 응답을 기대할 수 있습니다.

In [8]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(email)

Dear John,
I hope you are doing well.
I am writing this mail in response to the feedback that you provided us. I apologize for the
inconvenience that was caused to you.
I have shared your feedback with our customer support team. We work on improving ourselves and I
assure you that such incidents will not be repeated in the future.
I value you as a customer and I hope that you will give us another chance to serve you.
Thank you.
Yours truly,
Bob Smith
Service Manager.


## Summary

결론적으로 컨텍스트 없이 LLM을 호출하면 원하는 결과를 얻지 못할 수 있다는 사실을 알게 되었습니다. 컨텍스트를 추가하고 프롬프트 템플릿을 사용하여 LLM의 출력을 제한함으로써 원하는 출력을 성공적으로 얻을 수 있었습니다.